In [8]:


from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv(r'../venv/.env')
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY2")

In [2]:
#wiki tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wraper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max= 200)
wiki_tool = WikipediaQueryRun(api_wrapper= api_wraper)

In [3]:
# web loading

web_docs = WebBaseLoader("https://python.langchain.com/v0.2/docs/introduction/").load()
web_docs = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(web_docs)
vector_db = Chroma.from_documents(web_docs, OpenAIEmbeddings())
retriever = vector_db.as_retriever()
custom_tool = create_retriever_tool(retriever, "langsmith_retriever", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [4]:
# arxiv tool
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

ariv_wrapper = ArxivAPIWrapper(top_k_results= 1, doc_content_chars_max= 200)
arix_tool = ArxivQueryRun(api_wrapper= ariv_wrapper)
arix_tool.name 

'arxiv'

In [5]:
tools = [wiki_tool, custom_tool, arix_tool]

#### Agents

In [26]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt.messages


from langchain.agents import create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate
# prompt = ChatPromptTemplate.from_template("Use the given tools and answer the question using the context provided by tool {agent_scratchpad}, {chat_history}. Question {input}")

agent=create_openai_tools_agent(ChatOpenAI(),tools,prompt)

In [27]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [28]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [29]:
# agent_executor.invoke({"input":"which tool are you using? how do you decide which tool output you are going to use?"})
agent_executor.invoke({"input":"tell me about latest biomedical research"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'latest biomedical research'}`


Published: 2023-07-13
Title: A Survey for Biomedical Text Summarization: From Pre-trained to Large Language Models
Authors: Qianqian Xie, Zheheng Luo, Benyou Wang, Sophia Ananiadou
Summary: The exponeThe latest biomedical research includes a paper titled "A Survey for Biomedical Text Summarization: From Pre-trained to Large Language Models" by Qianqian Xie, Zheheng Luo, Benyou Wang, and Sophia Ananiadou. The paper discusses biomedical text summarization techniques, focusing on pre-trained and large language models.

> Finished chain.


{'input': 'tell me about latest biomedical research',
 'output': 'The latest biomedical research includes a paper titled "A Survey for Biomedical Text Summarization: From Pre-trained to Large Language Models" by Qianqian Xie, Zheheng Luo, Benyou Wang, and Sophia Ananiadou. The paper discusses biomedical text summarization techniques, focusing on pre-trained and large language models.'}